In [3]:
import pandas as pd 
import numpy as np
from tqdm import tqdm
import ctypes
import multiprocessing
from multiprocessing import Process, Manager, cpu_count

In [32]:
## 读取原始csv
data_raw = pd.read_csv("/home/anichikage/workspace/dram/dram-failure-prediction/data/raw/ali/mcelog.csv")


/home/anichikage/workspace/liulei/failure_prediction2


In [45]:
print(data_raw)

                   sid  memoryid  rankid  bankid     row  col  error_type  \
0             Server_1        10       1      13    3334  808           2   
1             Server_1        10       1      13    3334  808           2   
2             Server_1        10       1      13    3334  808           2   
3             Server_1        10       1      13    3334  808           2   
4             Server_1        10       1      13    3334  808           2   
...                ...       ...     ...     ...     ...  ...         ...   
75214687  Server_30500        16       0       8   51199  576           2   
75214688  Server_30501         0       0      12  109213  592           2   
75214689  Server_30502        20       1       7  120731  624           2   
75214690  Server_30502        20       1       7  120731  624           2   
75214691  Server_30502        20       1       7  120731  624           2   

                   error_time  
0         0001-06-14 06:40:59  
1         0

In [4]:
grouped = data_raw.groupby('sid') # 按sid将Excel分组
print(len(grouped))

30502


In [94]:
## 获取第一个分组
group_one = grouped.get_group("Server_2")
print(group_one)

           sid  memoryid  rankid  bankid     row   col  error_type  \
5     Server_2         3       1       1   25338  1016           1   
6     Server_2         3       1       1   17033  1016           1   
7     Server_2         3       1       1   29573  1016           1   
8     Server_2         3       1       1   29575  1016           1   
9     Server_2         3       1       1   29576  1016           1   
...        ...       ...     ...     ...     ...   ...         ...   
5517  Server_2         8       0       7   68884    32           2   
5518  Server_2         8       0       7   68884  1016           2   
5519  Server_2        20       1       3   39915   640           2   
5520  Server_2        20       1       3  105451   912           2   
5521  Server_2        20       1       3   39915   424           2   

               error_time  
5     0001-01-17 09:56:21  
6     0001-01-17 14:05:26  
7     0001-01-17 19:25:33  
8     0001-01-17 19:33:45  
9     0001-01-17 20

In [57]:
# 计算最大的 row和col数值
max_bank = max(data_raw["bankid"]) + 1 # 16 
max_row = max(data_raw["row"]) + 1 # 131072
max_col = max(data_raw["col"]) + 1 # 1017
print([max_bank, max_row, max_col]);

[16, 131072, 1017]


In [60]:
# 计算每个小块的行数和列数 
rows_per_block = int(np.ceil(max_row / 32))
cols_per_block = int(np.ceil(max_col / 16))

In [95]:
matrix_one = np.zeros((max_bank, max_row, max_col))
for _ , pos in group_one.iterrows():
    bank_index = pos["bankid"]
    row_index = pos["row"]
    col_index = pos["col"]
    
    matrix_one[bank_index, row_index, col_index] += 1 # 16x131071x1017 的大矩阵
    
# 压缩成16x32x16尺寸   
matrix_small = np.add.reduceat(
np.add.reduceat(matrix_one, np.arange(0, matrix_one.shape[1], rows_per_block), axis=1),# 起点、终点、步长
np.arange(0, matrix_one.shape[2], cols_per_block), axis=2)
    

In [1]:
print(matrix_small[7,:,:])

NameError: name 'matrix_small' is not defined

In [8]:
rows_per_block = 4096  # 行压缩块大小为 4096
cols_per_block = 64    # 列压缩块大小为 64

In [9]:
## 将row ,col存入一个大矩阵，再压缩成32x16
def process_group(group_One):
    matrix_one = np.zeros((max_row, max_col))
    for _, pos in group_One.iterrows():
        row_index = pos["row"]
        col_index = pos["col"]
        
        matrix_one[row_index, col_index] += 1 # 131071x1017 的大矩阵
        
    # 压缩成32x16尺寸   
    matrix_small = np.add.reduceat(
    np.add.reduceat(matrix_one, np.arange(0, matrix_one.shape[0], rows_per_block), axis=0),# 起点、终点、步长
    np.arange(0, matrix_one.shape[1], cols_per_block), axis=1)  
        
    return matrix_small

In [ ]:
def process_group_2(grouped, idx, result_dict):
    matrix_small = process_group(grouped.get_group(f"Server_{idx+1}"))
    result_dict[idx] = matrix_small
    
    

In [ ]:
group_one = grouped.get_group("Server_2")
print(process_group(group_one))

In [ ]:
## 循环处理第1~30502个group
matrix_all = np.empty((0,32,16))

for i in tqdm(range(30502)):
    group_one = grouped.get_group(f"Server_{i+1}")
    small_matrix = process_group(group_one)
    
    matrix_all =  np.concatenate((matrix_all, small_matrix[np.newaxis, :, :]), axis = 0)

In [11]:
# 创建共享数组 
array_shape = (30502, 32, 16)  # 30502个数组，每个数组大小为32x16
# 创建一个共享数组
shared_arr_base = multiprocessing.Array(ctypes.c_double, int(np.prod(array_shape)))
# 将共享数组转换为 NumPy 
shared_np_arr = np.frombuffer(shared_arr_base.get_obj(), dtype=np.float64).reshape(array_shape)


In [12]:
print(sum(sum(shared_np_arr[2017,:,:])))

0.0


In [13]:
def process_group_3(grouped, idx):
    matrix_small = process_group(grouped.get_group(f"Server_{idx+1}"))
    shared_np_arr[idx] = matrix_small
    

In [14]:
# 创建30个进程，每次处理1017个样本

# 同一个进程，同时能打开的文件数是有限的，一般是一千多，当时每次都是运行到2013就报错了
for idx in range(30): # tqdm是显示进度条
    processes = []
    for j in tqdm(range(idx*1017, (idx+1)*1017)): 
        p = Process(target = process_group_3, args = (grouped, j)) # 创建一个多进程任务
        processes.append(p) # 将进程添加到列表
        p.start() # 启动进程，开始执行process_one_agg(idx)
    for p in processes:
        p.join() # 等待进程完成

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 1011/1017 [03:18<00:01,  4.54it/s]Process Process-30503:
Traceback (most recent call last):
  File "/home/anichikage/anaconda3/envs/dram-py310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/anichikage/anaconda3/envs/dram-py310/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Process Process-30504:
Traceback (most recent call last):
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 1012/1017 [03:19<00:01,  4.56it/s]  File "/tmp/ipykernel_85345/388240735.py", line 2, in process_group_3
    matrix_small = process_group(grouped.get_group(f"Server_{idx+1}"))
  File "/home/anichikage/anaconda3/envs/dram-py310/lib/python3.10/multiprocessing/process.py", line 314, in _b

In [22]:
data_mce = shared_np_arr


In [27]:
np.save('data_processed/data_mce.npy', data_mce) 

In [28]:
x = np.load('data_processed/data_mce.npy') # 保存npy

In [ ]:
print(x[30491,:,:])

In [36]:
## 将数据进行放缩，归一化
data_mce = np.load("data_processed/data_mce.npy")

In [41]:
data_norm = 2 / (1 + np.exp(-data_mce)) - 1

In [ ]:
print(data_norm[30000,:,:])

In [45]:
2/(1+np.exp(-5))-1

0.9866142981514305

In [50]:
np.save('data_processed/data_norm.npy', data_norm)

In [4]:
## 读取原始csv
data_raw = pd.read_csv("/home/anichikage/workspace/dram/dram-failure-prediction/data/raw/ali/inventory.csv")
print(data_raw)

                sid server_manufacturer DRAM_model  DIMM_number
0          Server_1                  M2         C1           24
1          Server_2                  M2         C1           24
2          Server_3                  M2         A2           12
3          Server_4                  M2         A2           12
4          Server_5                  M2         A2           12
...             ...                 ...        ...          ...
30497  Server_30498                  M4         B1           12
30498  Server_30499                  M4         B1           12
30499  Server_30500                  M4         B1           12
30500  Server_30501                  M4         A1           12
30501  Server_30502                  M4         A1           12

[30502 rows x 4 columns]
